# **Configurando as bibliotecas do projeto**

## *Instalação de bibliotecas*

## *Importação de bibliotecas*

In [24]:
'''
    Essa célula será usada para a importação de bibliotecas.
'''

# Instala a biblioteca que será utilizada para se obter os dados financeiros.
import yfinance as yf
# Importa a biblioteca que será utilizada para a manipulação de DataFrames.
import pandas as pd
# Importa a biblioteca que será utilizada para lidar com objetos do tipo "datetime".
from datetime import datetime, timedelta
# Importa a biblioteca que será responsável pela tipagem de funções.
from typing import Union
# Importa a biblioteca que será utilizada para a manipulação de arrays e também para o uso de algumas funções matemáticas.
import numpy as np

# **Data Engineering**

## *Obtenção dos tickers + Configurações iniciais*

In [3]:
'''
    Essa célula será usada para se obter os tickets das companhias que fazem parte do S&P 500, com base em dados da wikipedia.
'''

# Salva em uma variável a url que contém a tabela com os tickets das companhias.
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

# Lê todas as tabelas presentes na url acima.
sp500_table = pd.read_html(url)

# Salva a coluna "Symbol" da primeira tabela em uma variável (tal coluna contém todos os tickets das companhias que fazem parte do S&P 500).
sp500_tickets = sp500_table[0]['Symbol']

# Transforma os tickets obtidos em uma lista.
sp500_tickets = sp500_tickets.tolist()

# Exibe a lista de tickets criada acima.
sp500_tickets

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ADBE',
 'AMD',
 'AES',
 'AFL',
 'A',
 'APD',
 'ABNB',
 'AKAM',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'AON',
 'APA',
 'AAPL',
 'AMAT',
 'APTV',
 'ACGL',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'AXON',
 'BKR',
 'BALL',
 'BAC',
 'BK',
 'BBWI',
 'BAX',
 'BDX',
 'BRK.B',
 'BBY',
 'BIO',
 'TECH',
 'BIIB',
 'BLK',
 'BX',
 'BA',
 'BKNG',
 'BWA',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF.B',
 'BLDR',
 'BG',
 'BXP',
 'CHRW',
 'CDNS',
 'CZR',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CTLT',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'COR',
 'CNC',
 'CNP',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA',
 'CAG'

In [4]:
'''
    Essa célula será usada para definir o intervalo de tempo dos dados que serão coletados, especificando a data de início e a data de fim.
'''

# Define a data inicial cujos dados serão coletados.
start_date = datetime(2018,1,1).date()
# Define a data final cujos dados serão coletados.
end_date = datetime(2024,1,1).date()

In [5]:
'''
    Essa célula será usada para criar um dicionário chamado setup, que contém os parâmetros principais para a coleta de dados, 
    incluindo a lista de tickers das companhias do S&P 500 e o intervalo de tempo.
'''

# Cria um dicionário para guardar alguns parâmetros importantes para a coleta de dados.
setup = {
    "tickets": sp500_tickets,
    "start_date": start_date,
    "end_date": end_date
}

## *Obtendo alguns dados via yfinance (Open, High, Low, Close, Adj Close, Volume)*

In [7]:
# Vamos usar os 10 tickers que estão presentes na variável de configuração setup.

# Cria uma variável que guardará a coluna "Adj Close" dos dados referentes a cada um dos tickers presentes na variável de configuração setup.
data_list = []

# Vamos reimportar os dados de cada um dos tickers. Perceba que estamos interessados apenas na coluna "Adj Close" dos dados importados.
for ticker in sp500_tickets[len(sp500_tickets)-10:]:
    # 
    ticker_data = yf.download(ticker, setup['start_date'], setup['end_date'])
    # Adiciono um nome a série temporal obtida acima.
    ticker_data.name=ticker
    # Salvo a série temporal em questão na variável "data_list".
    data_list.append(ticker_data)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


## *Feature Engineering*

### *Gerando os retornos aritméticos (diários, semanais e mensais) dos tickers*

In [31]:
def get_ticker_daily_arithmetic_returns(daily_adjusted_closing_prices: pd.Series) -> pd.DataFrame:
    '''
        Description:
            Essa função calcula e retorna a série temporal dos retornos aritméticos diários de um certo ticker.
        Args:
            daily_adjusted_closing_prices (pd.Series): Série temporal dos preços de fechamento diários ajustados do ticker em questão.
        Return:
            ticker_daily_arithmetic_returns (pd.Series): Série temporal dos retornos aritméticos diários do ticker em questão.
        Errors:
            TypeError: É esperado que o parâmetro "daily_adjusted_closing_prices" seja um objeto do tipo pd.Series.
            ValueError: É esperado que a série temporal "daily_adjusted_closing_prices" contenha apenas números.
    '''

    # Caso o parâmetro 'daily_adjusted_closing_prices' não seja um objeto do tipo pd.Series, um erro é retornado.
    if not isinstance(daily_adjusted_closing_prices, pd.Series):
        raise TypeError("É esperado que o parâmetro 'daily_adjusted_closing_prices' seja um objeto do tipo pd.Series.")

    # Caso a série temporal 'daily_adjusted_closing_prices' não seja numérica, um erro é retornado.
    if not(pd.api.types.is_numeric_dtype(daily_adjusted_closing_prices)):
        raise ValueError("É esperado que a série temporal 'daily_adjusted_closing_prices' contenha apenas números.")
    
    # Cria uma série temporal contendo os retornos diários do ticker em questão.
    ticker_daily_arithmetic_returns = daily_adjusted_closing_prices.pct_change()

    # Como não é possível calcular o retorno diário do "dia 0", removeremos a primeira linha da série temporal gerada acima.
    ticker_daily_arithmetic_returns = ticker_daily_arithmetic_returns[1:]
    
    # Retorna o DataFrame resultante.
    return ticker_daily_arithmetic_returns

In [39]:
def get_ticker_weekly_arithmetic_returns(daily_adjusted_closing_prices: pd.Series) -> pd.DataFrame:
    '''
        Description:
            Essa função calcula e retorna a série temporal dos retornos aritméticos semanais de um certo ticker.
        Args:
            daily_adjusted_closing_prices (pd.Series): Série temporal dos preços de fechamento diários ajustados do ticker em questão.
        Return:
            ticker_weekly_arithmetic_returns (pd.Series): Série temporal dos retornos aritméticos semanais do ticker em questão.
        Errors:
            TypeError: É esperado que o parâmetro "daily_adjusted_closing_prices" seja um objeto do tipo pd.Series.
            ValueError: É esperado que a série temporal "daily_adjusted_closing_prices" contenha apenas números.
    '''

    # Caso o parâmetro 'daily_adjusted_closing_prices' não seja um objeto do tipo pd.Series, um erro é retornado.
    if not isinstance(daily_adjusted_closing_prices, pd.Series):
        raise TypeError("É esperado que o parâmetro 'daily_adjusted_closing_prices' seja um objeto do tipo pd.Series.")

    # Caso a série temporal 'daily_adjusted_closing_prices' não seja numérica, um erro é retornado.
    if not(pd.api.types.is_numeric_dtype(daily_adjusted_closing_prices)):
        raise ValueError("É esperado que a série temporal 'daily_adjusted_closing_prices' contenha apenas números.")
    
    # Transforma a série temporal dos preços de fechamento diários na série temporal dos preços de fechamento semanais.
    weekly_adjusted_closing_prices = daily_adjusted_closing_prices.resample('W').last()
    
    # Cria uma série temporal contendo os retornos semanais do ticker em questão.
    ticker_weekly_arithmetic_returns = weekly_adjusted_closing_prices.pct_change()

    # Como não é possível calcular o retorno semanal da "semana 0", removeremos a primeira linha da série temporal gerada acima.
    ticker_weekly_arithmetic_returns = ticker_weekly_arithmetic_returns[1:]
    
    # Retorna o DataFrame resultante.
    return ticker_weekly_arithmetic_returns

In [43]:
def get_ticker_monthly_arithmetic_returns(daily_adjusted_closing_prices: pd.Series) -> pd.DataFrame:
    '''
        Description:
            Essa função calcula e retorna a série temporal dos retornos aritméticos mensais de um certo ticker.
        Args:
            daily_adjusted_closing_prices (pd.Series): Série temporal dos preços de fechamento diários ajustados do ticker em questão.
        Return:
            ticker_monthly_arithmetic_returns (pd.Series): Série temporal dos retornos aritméticos mensais do ticker em questão.
        Errors:
            TypeError: É esperado que o parâmetro "daily_adjusted_closing_prices" seja um objeto do tipo pd.Series.
            ValueError: É esperado que a série temporal "daily_adjusted_closing_prices" contenha apenas números.
    '''

    # Caso o parâmetro 'daily_adjusted_closing_prices' não seja um objeto do tipo pd.Series, um erro é retornado.
    if not isinstance(daily_adjusted_closing_prices, pd.Series):
        raise TypeError("É esperado que o parâmetro 'daily_adjusted_closing_prices' seja um objeto do tipo pd.Series.")

    # Caso a série temporal 'daily_adjusted_closing_prices' não seja numérica, um erro é retornado.
    if not(pd.api.types.is_numeric_dtype(daily_adjusted_closing_prices)):
        raise ValueError("É esperado que a série temporal 'daily_adjusted_closing_prices' contenha apenas números.")
    
    # Transforma a série temporal dos preços de fechamento diários na série temporal dos preços de fechamento mensais.
    monthly_adjusted_closing_prices = daily_adjusted_closing_prices.resample('M').last()
    
    # Cria uma série temporal contendo os retornos mensais do ticker em questão.
    ticker_monthly_arithmetic_returns = monthly_adjusted_closing_prices.pct_change()

    # Como não é possível calcular o retorno mensal no "mês 0", removeremos a primeira linha da série temporal gerada acima.
    ticker_monthly_arithmetic_returns = ticker_monthly_arithmetic_returns[1:]
    
    # Retorna o DataFrame resultante.
    return ticker_monthly_arithmetic_returns

### *Gerando os retornos logarítmicos (diários, semanais e mensais) dos tickers*

In [32]:
def get_ticker_daily_logarithmic_returns(daily_adjusted_closing_prices: pd.Series) -> pd.DataFrame:
    '''
        Description:
            Essa função calcula e retorna a série temporal dos retornos logarítmicos diários de um certo ticker.
        Args:
            daily_adjusted_closing_prices (pd.Series): Série temporal dos preços de fechamento diários ajustados do ticker em questão.
        Return:
            ticker_daily_arithmetic_returns (pd.Series): Série temporal dos retornos logarítmicos diários do ticker em questão.
        Errors:
            TypeError: É esperado que o parâmetro "daily_adjusted_closing_prices" seja um objeto do tipo pd.Series.
            ValueError: É esperado que a série temporal "daily_adjusted_closing_prices" contenha apenas números.
    '''

    # Caso o parâmetro 'daily_adjusted_closing_prices' não seja um objeto do tipo pd.Series, um erro é retornado.
    if not isinstance(daily_adjusted_closing_prices, pd.Series):
        raise TypeError("É esperado que o parâmetro 'daily_adjusted_closing_prices' seja um objeto do tipo pd.Series.")
    
    # Caso a série temporal 'daily_adjusted_closing_prices' não seja numérica, um erro é retornado.
    if not(pd.api.types.is_numeric_dtype(daily_adjusted_closing_prices)):
        raise ValueError("É esperado que a série temporal 'daily_adjusted_closing_prices' contenha apenas números.")
    
    # Obtem a série temporal dos retornos diários do ticker em questão
    ticker_daily_arithmetic_returns = get_ticker_daily_arithmetic_returns(daily_adjusted_closing_prices)
    
    # Transforma a série temporal dos retornos diários do ticker em questão em sua versão logarítmica.
    ticker_daily_logarithmic_returns = ticker_daily_arithmetic_returns.apply(lambda x: np.log(1 + x))
    
    # Retorna o DataFrame resultante.
    return ticker_daily_logarithmic_returns

In [38]:
def get_ticker_weekly_logarithmic_returns(daily_adjusted_closing_prices: pd.Series) -> pd.DataFrame:
    '''
        Description:
            Essa função calcula e retorna a série temporal dos retornos logarítmicos semanais de um certo ticker.
        Args:
            daily_adjusted_closing_prices (pd.Series): Série temporal dos preços de fechamento diários ajustados do ticker em questão.
        Return:
            ticker_weekly_logarithmic_returns (pd.Series): Série temporal dos retornos logarítmicos semanais do ticker em questão.
        Errors:
            TypeError: É esperado que o parâmetro "daily_adjusted_closing_prices" seja um objeto do tipo pd.Series.
            ValueError: É esperado que a série temporal "daily_adjusted_closing_prices" contenha apenas números.
    '''

    # Caso o parâmetro 'daily_adjusted_closing_prices' não seja um objeto do tipo pd.Series, um erro é retornado.
    if not isinstance(daily_adjusted_closing_prices, pd.Series):
        raise TypeError("É esperado que o parâmetro 'daily_adjusted_closing_prices' seja um objeto do tipo pd.Series.")

    # Caso a série temporal 'daily_adjusted_closing_prices' não seja numérica, um erro é retornado.
    if not(pd.api.types.is_numeric_dtype(daily_adjusted_closing_prices)):
        raise ValueError("É esperado que a série temporal 'daily_adjusted_closing_prices' contenha apenas números.")
    
    # Obtem a série temporal dos retornos semanais do ticker em questão
    ticker_weekly_arithmetic_returns = get_ticker_weekly_arithmetic_returns(daily_adjusted_closing_prices)
    
    # Transforma a série temporal dos retornos semanais do ticker em questão em sua versão logarítmica.
    ticker_weekly_logarithmic_returns = ticker_weekly_arithmetic_returns.apply(lambda x: np.log(1 + x))
    
    # Retorna o DataFrame resultante.
    return ticker_weekly_logarithmic_returns

In [45]:
def get_ticker_monthly_logarithmic_returns(daily_adjusted_closing_prices: pd.Series) -> pd.DataFrame:
    '''
        Description:
            Essa função calcula e retorna a série temporal dos retornos logarítmicos mensais de um certo ticker.
        Args:
            daily_adjusted_closing_prices (pd.Series): Série temporal dos preços de fechamento diários ajustados do ticker em questão.
        Return:
            ticker_weekly_logarithmic_returns (pd.Series): Série temporal dos retornos logarítmicos mensais do ticker em questão.
        Errors:
            TypeError: É esperado que o parâmetro "daily_adjusted_closing_prices" seja um objeto do tipo pd.Series.
            ValueError: É esperado que a série temporal "daily_adjusted_closing_prices" contenha apenas números.
    '''

    # Caso o parâmetro 'daily_adjusted_closing_prices' não seja um objeto do tipo pd.Series, um erro é retornado.
    if not isinstance(daily_adjusted_closing_prices, pd.Series):
        raise TypeError("É esperado que o parâmetro 'daily_adjusted_closing_prices' seja um objeto do tipo pd.Series.")

    # Caso a série temporal 'daily_adjusted_closing_prices' não seja numérica, um erro é retornado.
    if not(pd.api.types.is_numeric_dtype(daily_adjusted_closing_prices)):
        raise ValueError("É esperado que a série temporal 'daily_adjusted_closing_prices' contenha apenas números.")
    
    # Obtem a série temporal dos retornos semanais do ticker em questão
    ticker_monthly_arithmetic_returns = get_ticker_monthly_arithmetic_returns(daily_adjusted_closing_prices)
    
    # Transforma a série temporal dos retornos semanais do ticker em questão em sua versão logarítmica.
    ticker_monthly_logarithmic_returns = ticker_monthly_arithmetic_returns.apply(lambda x: np.log(1 + x))
    
    # Retorna o DataFrame resultante.
    return ticker_monthly_logarithmic_returns

### *Gerando as volatilidades históricas (diárias, semanais e mensais) dos tickers*